In [ ]:
from helper import * 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

path = 'data'
df = load_data(f'{path}/driving_log.csv')
df = balance_data(df, 'Steering')
df

The car go straight most of the time that makes steering values around 0 extremely higher than the other. Drop the steering data to balance the whole dataset. 

In [ ]:
img_path, steering = load_data_to_array(path, df)
# fixed value for random_state to keep train and test datasets same each time
X_train, X_val, Y_train, Y_val = train_test_split(img_path, steering, test_size=0.2, random_state=1) 
X_train, Y_train = img_preprocess_pipeline(X_train, Y_train, 1)
X_val, Y_val = img_preprocess_pipeline(X_val, Y_val, 0 )

print(X_train.shape)
print(Y_train.shape)
print("Training imgs: ", len(X_train))
print("Test imgs: ", len(X_val))

In [ ]:
import wandb
from wandb.keras import WandbCallback
from wandb_config import sweep_config

wandb.init(project='cnn_thesis')

sweep_id = wandb.sweep(sweep_config, project='cnn_thesis')
log_callback = WandbCallback(log_evaluation=True)

In [ ]:

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        optim = build_optimizer(config.optimizer, config.learning_rate)
        model = build_network(activation=config.activation, optimizer=optim)
        
        model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=config.batch_size,
                        verbose=1, epochs=config.epochs, callbacks=[log_callback])

In [ ]:
wandb.agent(sweep_id, train, count=20)

In [ ]:
''' 
Iterations: number of batches to complete one epoch 
Batches: number of training samples in one iteration 
Steps per epoch = number of training sample // batch size
'''
